# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 3:20PM,243813,10,0085930698,ISDIN Corporation,Released
1,Aug 9 2022 3:20PM,243813,10,0085930704,ISDIN Corporation,Released
2,Aug 9 2022 3:20PM,243813,10,0085930718,ISDIN Corporation,Released
3,Aug 9 2022 3:20PM,243813,10,0085930726,ISDIN Corporation,Released
4,Aug 9 2022 3:20PM,243813,10,0085930736,ISDIN Corporation,Released
5,Aug 9 2022 3:20PM,243813,10,0085930733,ISDIN Corporation,Released
6,Aug 9 2022 3:20PM,243813,10,0085930734,ISDIN Corporation,Released
7,Aug 9 2022 3:17PM,243812,10,8381893,Yusen – 3D Matrix,Released
8,Aug 9 2022 3:15PM,243811,10,SONSB0001894,"Nextsource Biotechnology, LLC",Released
9,Aug 9 2022 3:15PM,243811,10,SONSB0001893,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,243809,Released,2
37,243810,Released,1
38,243811,Released,3
39,243812,Released,1
40,243813,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243809,NaN,NaN,2.0
243810,NaN,NaN,1.0
243811,NaN,NaN,3.0
243812,NaN,NaN,1.0
243813,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243722,0.0,0.0,1.0
243723,33.0,15.0,60.0
243732,0.0,0.0,1.0
243744,3.0,12.0,0.0
243745,0.0,14.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243722,0,0,1
243723,33,15,60
243732,0,0,1
243744,3,12,0
243745,0,14,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243722,0,0,1
1,243723,33,15,60
2,243732,0,0,1
3,243744,3,12,0
4,243745,0,14,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243722,,,1
1,243723,33,15,60
2,243732,,,1
3,243744,3,12,
4,243745,,14,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation
7,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix
8,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC"
11,Aug 9 2022 3:13PM,243810,21,"NBTY Global, Inc."
12,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc."
67,Aug 9 2022 3:00PM,243809,10,Emerginnova
69,Aug 9 2022 2:56PM,243807,10,Yusen – 3D Matrix
70,Aug 9 2022 2:51PM,243805,20,Alumier Labs Inc.
85,Aug 9 2022 2:50PM,243769,20,Alumier Labs Inc.
94,Aug 9 2022 2:44PM,243804,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,,,7
1,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,,,1
2,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",,,3
3,Aug 9 2022 3:13PM,243810,21,"NBTY Global, Inc.",,,1
4,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",,,55
5,Aug 9 2022 3:00PM,243809,10,Emerginnova,,,2
6,Aug 9 2022 2:56PM,243807,10,Yusen – 3D Matrix,,,1
7,Aug 9 2022 2:51PM,243805,20,Alumier Labs Inc.,,,15
8,Aug 9 2022 2:50PM,243769,20,Alumier Labs Inc.,,,9
9,Aug 9 2022 2:44PM,243804,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,7,,
1,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,1,,
2,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",3,,
3,Aug 9 2022 3:13PM,243810,21,"NBTY Global, Inc.",1,,
4,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",55,,
5,Aug 9 2022 3:00PM,243809,10,Emerginnova,2,,
6,Aug 9 2022 2:56PM,243807,10,Yusen – 3D Matrix,1,,
7,Aug 9 2022 2:51PM,243805,20,Alumier Labs Inc.,15,,
8,Aug 9 2022 2:50PM,243769,20,Alumier Labs Inc.,9,,
9,Aug 9 2022 2:44PM,243804,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,7,,
1,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,1,,
2,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",3,,
3,Aug 9 2022 3:13PM,243810,21,"NBTY Global, Inc.",1,,
4,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",55,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,7.0,NaN,NaN
1,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
3,Aug 9 2022 3:13PM,243810,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",55.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,7.0,0.0,0.0
1,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
3,Aug 9 2022 3:13PM,243810,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",55.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3413051,52.0,17.0,0.0
12,243783,0.0,1.0,0.0
15,487478,68.0,15.0,33.0
19,487563,18.0,14.0,13.0
20,1706470,126.0,16.0,4.0
21,487532,2.0,0.0,0.0
22,487606,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3413051,52.0,17.0,0.0
1,12,243783,0.0,1.0,0.0
2,15,487478,68.0,15.0,33.0
3,19,487563,18.0,14.0,13.0
4,20,1706470,126.0,16.0,4.0
5,21,487532,2.0,0.0,0.0
6,22,487606,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,17.0,0.0
1,12,0.0,1.0,0.0
2,15,68.0,15.0,33.0
3,19,18.0,14.0,13.0
4,20,126.0,16.0,4.0
5,21,2.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,12,Released,0.0
2,15,Released,68.0
3,19,Released,18.0
4,20,Released,126.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,33.0,13.0,4.0,0.0,0.0
Executing,17.0,1.0,15.0,14.0,16.0,0.0,0.0
Released,52.0,0.0,68.0,18.0,126.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,33.0,13.0,4.0,0.0,0.0
1,Executing,17.0,1.0,15.0,14.0,16.0,0.0,0.0
2,Released,52.0,0.0,68.0,18.0,126.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,0.0,0.0,33.0,13.0,4.0,0.0,0.0
1,Executing,17.0,1.0,15.0,14.0,16.0,0.0,0.0
2,Released,52.0,0.0,68.0,18.0,126.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()